In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [2]:
import os 
import math
import tensorflow as tf
from pathlib import Path
from src.metrics.losses import WeightedBinaryCrossentropy, SimpleWeightedBinaryCrossentropy
from src.metrics.metrics import F2Score
import keras.backend.tensorflow_backend as K
from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture


Using TensorFlow backend.


In [3]:
# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:42:00.0, compute capability: 7.5



In [4]:
y_pred = tf.constant([[0.,1.,0.], [0.,0.25,1.], [1.,0.,0.75]])
y_true = tf.constant([[0,1.0,1.0], [1.0,0,1.0], [1.,1.,0]])
weights = tf.constant([2, 0.5, 10.])
print(y_pred)
print(y_true)
print(weights)

tf.Tensor(
[[0.   1.   0.  ]
 [0.   0.25 1.  ]
 [1.   0.   0.75]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 0.]], shape=(3, 3), dtype=float32)
tf.Tensor([ 2.   0.5 10. ], shape=(3,), dtype=float32)


In [5]:
# sanity checks
# 1. throw all classes at the same time in backend cross_entropy
print("All classes in backend cross_entropy")
print(K.binary_crossentropy(y_true, y_pred))
print(K.mean(K.binary_crossentropy(y_true, y_pred)))
print()

# 2. throw all classes at the same time in keras loss cross_entropy
print("All classes in keras loss cross_entropy")
print(tf.keras.losses.BinaryCrossentropy()(y_true, y_pred))
print()

# 3. apply backend cross entropy for each class seperately:
print("backend cross entropy for each class seperately:")
print(K.mean(K.binary_crossentropy(y_true, y_pred), axis=1))
print()

for i in range(len(y_pred)):
    #print("y_pred", y_pred[:, i])
    #print("y_true", y_true[:, i])
    print("cross_entropy for class ", i)
    print(tf.keras.losses.BinaryCrossentropy()(y_true[:, i], y_pred[:, i]))
    print()
    
# 4. check simple custom weighted cross entropy with weights set to 1
print("simple custom weighted cross entrop")
simple_weighted_bce_fn = SimpleWeightedBinaryCrossentropy(tf.constant([1., 1., 1.]))
print(simple_weighted_bce_fn(y_true, y_pred))
print()

# 5. check custom weighted cross entropy with pos and negative weights set to 1
print("custom weighted cross entrop")
weighted_bce_fn = WeightedBinaryCrossentropy(tf.constant([1., 1., 1.]),  tf.constant([1., 1., 1.]))
print(weighted_bce_fn(y_true, y_pred))
print()


All classes in backend cross_entropy
tf.Tensor(
[[-0.        -0.        15.424949 ]
 [15.424949   0.2876819 -0.       ]
 [-0.        15.424949   1.386294 ]], shape=(3, 3), dtype=float32)
tf.Tensor(5.3276467, shape=(), dtype=float32)

All classes in keras loss cross_entropy
tf.Tensor(5.3276467, shape=(), dtype=float32)

backend cross entropy for each class seperately:
tf.Tensor([5.1416497 5.2375436 5.603748 ], shape=(3,), dtype=float32)

cross_entropy for class  0
tf.Tensor(5.1416497, shape=(), dtype=float32)

cross_entropy for class  1
tf.Tensor(5.2375436, shape=(), dtype=float32)

cross_entropy for class  2
tf.Tensor(5.603748, shape=(), dtype=float32)

simple custom weighted cross entrop
tf.Tensor(5.3276467, shape=(), dtype=float32)

custom weighted cross entrop
tf.Tensor(5.3276467, shape=(), dtype=float32)



In [6]:
metrics = [tf.keras.metrics.AUC(multi_label=True, name="auc"),
           tf.keras.metrics.Precision(name="precision"),
           tf.keras.metrics.Recall(name="recall"),
           F2Score(name="f2_score"),
           tf.keras.metrics.BinaryAccuracy(name="binary_accuracy")
          ]

chexpert_columns = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion',
                       'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax',
                       'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

chexpert_benchmark = Benchmark(Path(os.environ.get("CHEXPERT_DEV_DATASET_DIRECTORY")),
                                  chexpert_columns,
                                  "Chexpert dev small batches",
                                  epochs=10,
                                  train_labels="train.csv",
                                  path_column="Path",
                                  split_group='patient_id',
                                  batch_size=16,
                                  metrics = metrics,
                                  loss = WeightedBinaryCrossentropy(tf.constant([1.,]*len(chexpert_columns)))
                                  #loss = weighted_binary_cross_entropy(tf.constant([1., 1., 1.]))
                              )
chexpert_benchmark.summary()

"The benchmark was initialized for the chexpert_dev dataset with batch size of 16, shuffel set to True and images rescaled to dimension (256, 256).\nThe training was done for 10 epochs using the Adam optimizer and <src.metrics.losses.WeightedBinaryCrossentropy object at 0x7f0e70803b00> loss.\nA total of 14 labels/pathologies were included in the training and encoded using the 'uzeroes' method.\nThe traing set included 9742 number of sample, the validation set 2402, and the test set 3470. "

In [7]:
model_resnet_chexpert = SimpleBaseArchitecture(ResNet152V2, len(chexpert_columns))

In [8]:
experiment_resnet_chexpert = Experiment(chexpert_benchmark, model_resnet_chexpert, "ResNet152V2_CheXpert_dev_16")

In [9]:
train_results = experiment_resnet_chexpert.train()

Epoch 1/10
  1/608 [..............................] - ETA: 0s - loss: 0.7318 - auc: 0.3390 - precision: 0.1468 - recall: 0.5000 - f2_score: 0.3376 - binary_accuracy: 0.5134WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
248/608 [===========>..................] - ETA: 33s - loss: 0.3559 - auc: 0.6089 - precision: 0.5834 - recall: 0.3506 - f2_score: 0.3810 - binary_accuracy: 0.8525

KeyboardInterrupt: 

In [35]:
import numpy as np
np.unique()[1:]

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14.])

In [41]:
sparse = chexpert_benchmark.traingen.get_labels() * (np.array(range(14)) + 1)

array([ 9., 11., 14., ...,  6., 11.,  1.])

In [64]:
def compute_class_weight(datagenerator):
    labels = datagenerator.get_labels()
    num_samples, num_classes = labels.shape
    class_weights_positive = [0,]*num_classes
    class_weights_negative = [0,]*num_classes
    # we encode all classes with their position +1
    # that way 0s are negatives and any positive number is the positive occurance of this class
    sparse_positive_labels = labels * (np.array(range(num_classes)) + 1)
    # here we encode all negative occurances with their class id
    sparse_negativ_labels = (1-labels) * (np.array(range(num_classes)) + 1)
    for i in range(num_classes):
        class_id = i +1
        num_positive_occurence = len(sparse_positive_labels[sparse_positive_labels == class_id])
        num_negative_occurence = len(sparse_negativ_labels[sparse_negativ_labels == class_id])
        class_weights_positive[i] = (num_positive_occurence+num_negative_occurence) / num_positive_occurence
        class_weights_negative[i] = (num_positive_occurence+num_negative_occurence) / num_negative_occurence
    return class_weights_positive,class_weights_negative


In [57]:
compute_class_weight(chexpert_benchmark.traingen)

[9.776884422110554,
 19.732251521298174,
 8.195450716090985,
 2.088001717106675,
 22.782201405152225,
 4.391873589164786,
 15.105590062111801,
 36.1635687732342,
 6.387393302692055,
 11.06712172923777,
 2.671793463334249,
 50.931937172774866,
 26.579234972677597,
 1.9748274462038165]

In [65]:
compute_class_weight(chexpert_benchmark.traingen)

([9.776884422110554,
  19.732251521298174,
  8.195450716090985,
  2.088001717106675,
  22.782201405152225,
  4.391873589164786,
  15.105590062111801,
  36.1635687732342,
  6.387393302692055,
  11.06712172923777,
  2.671793463334249,
  50.931937172774866,
  26.579234972677597,
  1.9748274462038165],
 [1.1139356463987176,
  1.053383865728208,
  1.1389767006205362,
  1.9191161964884593,
  1.0459090420384904,
  1.2948223079994676,
  1.0708938793483047,
  1.0284385241568876,
  1.1856185252894575,
  1.0993332579952537,
  1.5981600131427633,
  1.0200272622417952,
  1.0390942106387524,
  2.02582257392753])

In [67]:
sparse_negativ_labels = (1 - chexpert_benchmark.traingen.get_labels()) * (np.array(range(14)) + 1)

In [69]:
 (chexpert_benchmark.traingen.get_labels().shape[0]) /  len(sparse_negativ_labels[sparse_negativ_labels == 1])

1.1139356463987176

In [75]:
np.array([np.float("nan"),2,3]) * np.array(range(3)) + 1

array([nan,  3.,  7.])